In [3]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from gymnasium.wrappers import TimeLimit
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from deform_rl.envs.Rectangle_env.environment import Rectangle1D
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import EvalCallback

import pygame

In [4]:
def make_env(rank,seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = Rectangle1D(render_mode='human',oneD=False)
        env = TimeLimit(env,max_episode_steps=1000)
        env = Monitor(env)
        # use a seed for reproducibility
        # Important: use a different seed for each environment
        # otherwise they would generate the same experiences
        env.reset(seed=seed + rank)
        return env

    set_random_seed(seed)
    return _init



env0 = DummyVecEnv([make_env(i+4) for i in range(4)])
training_env =VecNormalize(env0)
env1 = DummyVecEnv([make_env(i+4) for i in range(1)])
validation_env = VecNormalize(env1)

save_dir = os.path.join("saved_models")
log_dir = os.path.join("logs")
os.makedirs(save_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)



In [5]:
model = PPO('MlpPolicy', training_env,device='cpu',verbose=1,tensorboard_log=log_dir)
model.learn(800000,tb_log_name='rectangle_v2') 
model.save("shit_model")
training_env.save("shitNormalize.pkl")


Using cpu device
Logging to logs/rectangle_v2_7
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 912       |
|    ep_rew_mean     | -6.33e+03 |
| time/              |           |
|    fps             | 3322      |
|    iterations      | 1         |
|    time_elapsed    | 2         |
|    total_timesteps | 8192      |
----------------------------------


KeyboardInterrupt: 

In [4]:
# model.learn(800000,tb_log_name='rectangle_v2',reset_num_timesteps=False) 
# model.save("shit_model")
# training_env.save("shitNormalize.pkl")


In [15]:
env2 = DummyVecEnv([make_env(i+4) for i in range(1)])
testing_env = VecNormalize.load('shitNormalize.pkl',env2)
testing_env.training = False
# testing_env = env2
model = PPO.load('shit_model',testing_env)

In [16]:
obs = testing_env.reset()
cum_reward = cnt=0
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    
    obs, reward, done, info = testing_env.step(action)
    cum_reward += reward
    cnt +=1
    if done:
        obs = testing_env.reset()
        print("Episode done: ", cnt, "reward: ", cum_reward)
        
        cnt=0
        cum_reward=0
    testing_env.render()
    if pygame.event.get(pygame.QUIT):
        break
    # print(obs, reward)
testing_env.close()

Episode done:  94 reward:  [0.92547663]
Episode done:  73 reward:  [1.1757298]
Episode done:  63 reward:  [1.29802106]
Episode done:  72 reward:  [1.18828288]
Episode done:  58 reward:  [1.35933229]
Episode done:  66 reward:  [1.260598]
Episode done:  76 reward:  [1.13560119]
Episode done:  57 reward:  [1.37404562]
Episode done:  112 reward:  [0.71518963]
Episode done:  50 reward:  [1.45691634]
Episode done:  44 reward:  [1.53313795]
Episode done:  84 reward:  [1.04751599]
Episode done:  85 reward:  [1.02925843]


Episode done:  95 reward:  [214.71915]
Episode done:  74 reward:  [271.5203]
Episode done:  65 reward:  [299.72574]
Episode done:  73 reward:  [274.79398]
Episode done:  59 reward:  [316.1392]
Episode done:  67 reward:  [293.01675]
Episode done:  77 reward:  [264.3034]
Episode done:  58 reward:  [318.39868]
Episode done:  113 reward:  [164.8003]
Episode done:  51 reward:  [338.93597]
Episode done:  45 reward:  [356.6471]
Episode done:  85 reward:  [242.5853]
Episode done:  87 reward:  [238.1221]